In [2]:
!mkdir ~/.kaggle
!ls ~/.kaggle

In [3]:
! cp "/content/drive/MyDrive/Colab Notebooks/kaggle.json" ~/.kaggle/

In [4]:
! kaggle datasets download varpit94/tesla-stock-data-updated-till-28jun2021

  0% 0.00/58.6k [00:00<?, ?B/s]
100% 58.6k/58.6k [00:00<00:00, 43.5MB/s]


In [5]:
!unzip "/content/tesla-stock-data-updated-till-28jun2021.zip"

Archive:  /content/tesla-stock-data-updated-till-28jun2021.zip
  inflating: TSLA.csv                


In [29]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt


In [30]:
df = pd.read_csv("/content/TSLA.csv")
df = df.drop(axis=1,columns=["Date"])
df

,Open,High,Low,Close,Adj Close,Volume
0,3.800000,5.000000,3.508000,4.778000,4.778000,93831500
1,5.158000,6.084000,4.660000,4.766000,4.766000,85935500
2,5.000000,5.184000,4.054000,4.392000,4.392000,41094000
3,4.600000,4.620000,3.742000,3.840000,3.840000,25699000
4,4.000000,4.000000,3.166000,3.222000,3.222000,34334500
...,...,...,...,...,...,...
2951,874.489990,907.849976,867.390015,905.390015,905.390015,33408500
2952,914.979980,942.849976,907.090027,921.159973,921.159973,27327200
2953,930.000000,997.859985,921.750000,993.979980,993.979980,35289500
2954,979.940002,1040.699951,976.400024,999.109985,999.109985,40225400


In [31]:
df = df[["High",	"Low", 	"Close",	"Adj Close",	"Volume","Open"]]
df

,High,Low,Close,Adj Close,Volume,Open
0,5.000000,3.508000,4.778000,4.778000,93831500,3.800000
1,6.084000,4.660000,4.766000,4.766000,85935500,5.158000
2,5.184000,4.054000,4.392000,4.392000,41094000,5.000000
3,4.620000,3.742000,3.840000,3.840000,25699000,4.600000
4,4.000000,3.166000,3.222000,3.222000,34334500,4.000000
...,...,...,...,...,...,...
2951,907.849976,867.390015,905.390015,905.390015,33408500,874.489990
2952,942.849976,907.090027,921.159973,921.159973,27327200,914.979980
2953,997.859985,921.750000,993.979980,993.979980,35289500,930.000000
2954,1040.699951,976.400024,999.109985,999.109985,40225400,979.940002


In [32]:
def to_time_series(target_df,n_rows,n_target):
  # Create a new DataFrame to store the flattened rows
  df = target_df.copy()
  new_df = pd.DataFrame(columns=[f'x{i}' for i in range(1, len(df.columns) * n_rows + 1)] + [f'y{j}' for j in range(1,n_target + 1)])
  # Recursive process
  while len(df) >= (n_rows + 1):
    # Flatten the first 60 rows into one row
    flattened_row = df.head(n_rows).values.flatten()
    # Append the last column of the 61st row
    flattened_row = np.append(flattened_row, df.iloc[n_rows, -n_target:])
    # Add this row to the new DataFrame
    new_df.loc[len(new_df)] = flattened_row
    # Remove the first row of the old DataFrame
    df = df.iloc[1:]
  return new_df

df = to_time_series(df,60,1)
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x352,x353,x354,x355,x356,x357,x358,x359,x360,y1
0,5.000000,3.508000,4.778000,4.778000,93831500.0,3.800000,6.084000,4.660000,4.766000,4.766000,...,4.154000,3980000.0,4.17800,4.190000,3.960000,3.974000,3.974000,4814500.0,4.174000,3.978000
1,6.084000,4.660000,4.766000,4.766000,85935500.0,5.158000,5.184000,4.054000,4.392000,4.392000,...,3.974000,4814500.0,4.17400,4.028000,3.900000,3.912000,3.912000,3340500.0,3.978000,3.990000
2,5.184000,4.054000,4.392000,4.392000,41094000.0,5.000000,4.620000,3.742000,3.840000,3.840000,...,3.912000,3340500.0,3.97800,4.038000,3.930000,4.020000,4.020000,2894500.0,3.990000,4.080000
3,4.620000,3.742000,3.840000,3.840000,25699000.0,4.600000,4.000000,3.166000,3.222000,3.222000,...,4.020000,2894500.0,3.99000,4.162000,4.010000,4.106000,4.106000,2093000.0,4.080000,4.208000
4,4.000000,3.166000,3.222000,3.222000,34334500.0,4.000000,3.326000,2.996000,3.160000,3.160000,...,4.106000,2093000.0,4.08000,4.298000,4.152000,4.280000,4.280000,6072500.0,4.208000,4.238000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891,939.500000,886.119995,938.530029,938.530029,23839300.0,916.869995,1015.659973,957.049988,1008.869995,1008.869995,...,840.229980,28009600.0,809.00000,875.000000,825.719971,871.599976,871.599976,22194300.0,830.989990,874.489990
2892,1015.659973,957.049988,1008.869995,1008.869995,31211400.0,965.659973,1072.979980,997.559998,1067.000000,1067.000000,...,871.599976,22194300.0,830.98999,907.849976,867.390015,905.390015,905.390015,33408500.0,874.489990,914.979980
2893,1072.979980,997.559998,1067.000000,1067.000000,30904400.0,1006.799988,1117.000000,1070.719971,1093.939941,1093.939941,...,905.390015,33408500.0,874.48999,942.849976,907.090027,921.159973,921.159973,27327200.0,914.979980,930.000000
2894,1117.000000,1070.719971,1093.939941,1093.939941,23715300.0,1073.670044,1119.000000,1078.420044,1088.469971,1088.469971,...,921.159973,27327200.0,914.97998,997.859985,921.750000,993.979980,993.979980,35289500.0,930.000000,979.940002


In [33]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Apply Min-Max scaling to each row
scaled_rows = scaler.fit_transform(df)

# Convert the scaled rows back to a DataFrame
df = pd.DataFrame(scaled_rows, columns=df.columns)
df

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x352,x353,x354,x355,x356,x357,x358,x359,x360,y1
0,-0.523086,-0.529726,-0.523823,-0.523823,2.210692,-0.528303,-0.518406,-0.524483,-0.523901,-0.523901,...,-0.544883,-0.990901,-0.544640,-0.544691,-0.546325,-0.545896,-0.545896,-0.961693,-0.544976,-0.546041
1,-0.518356,-0.524466,-0.523876,-0.523876,1.931064,-0.522248,-0.522321,-0.527241,-0.525561,-0.525561,...,-0.545600,-0.961098,-0.544656,-0.545321,-0.546570,-0.546143,-0.546143,-1.014342,-0.545756,-0.545993
2,-0.522283,-0.527233,-0.525542,-0.525542,0.343053,-0.522952,-0.524774,-0.528661,-0.528012,-0.528012,...,-0.545847,-1.013739,-0.545437,-0.545282,-0.546447,-0.545713,-0.545713,-1.030273,-0.545708,-0.545636
3,-0.524744,-0.528658,-0.528001,-0.528001,-0.202143,-0.524736,-0.527471,-0.531283,-0.530756,-0.530756,...,-0.545417,-1.029668,-0.545390,-0.544800,-0.546121,-0.545371,-0.545371,-1.058901,-0.545350,-0.545127
4,-0.527450,-0.531288,-0.530754,-0.530754,0.103673,-0.527411,-0.530403,-0.532057,-0.531032,-0.531032,...,-0.545074,-1.058292,-0.545031,-0.544271,-0.545541,-0.544679,-0.544679,-0.916759,-0.544840,-0.545008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891,3.554544,3.500326,3.635344,3.635344,-0.268002,3.543292,3.873208,3.810493,3.934684,3.934684,...,2.787501,-0.132724,2.663830,2.841379,2.807749,2.905452,2.905452,-0.340907,2.744995,2.910858
2892,3.886863,3.824196,3.948656,3.948656,-0.006928,3.760858,4.122548,3.994882,4.192802,4.192802,...,2.912534,-0.340408,2.751494,2.969113,2.977828,3.039866,3.039866,0.059650,2.918085,3.071648
2893,4.136975,4.009166,4.207582,4.207582,-0.017800,3.944311,4.314033,4.327883,4.312425,4.312425,...,3.047212,0.060088,2.924909,3.105208,3.139866,3.102598,3.102598,-0.157566,3.079198,3.131295
2894,4.329054,4.343219,4.327579,4.327579,-0.272393,4.242501,4.322733,4.362931,4.288136,4.288136,...,3.110067,-0.157095,3.086325,3.319110,3.199702,3.392270,3.392270,0.126837,3.138964,3.329612


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Assuming X contains features and y contains target variable
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Choose a linear regression model
model = LinearRegression()

# Step 4: Train the model on the training data
model.fit(X_train, y_train)

# Step 5: Evaluate the model
# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model's performance using metrics such as mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calculate R-squared (R2) score
r2 = r2_score(y_test, y_pred)
print("R-squared (R2) Score:", r2)

Mean Squared Error: 0.0011567491245111465
R-squared (R2) Score: 0.9989476553093379
